In [1]:
import os
import sys
sys.path.append(os.path.expandvars("$HOME/NEXT/NEXT100-0nubb-analysis"))
import glob
import pandas as pd
import numpy  as np
import tables as tb

%matplotlib notebook
import matplotlib.pyplot as plt

from invisible_cities.io.dst_io import load_dst, load_dsts

from fitting_tools.experiment_generator import generate_background_experiment, create_unique_event_number

plt.rcParams["font.size"]      = 15
plt.rcParams["font.family"]    = "sans-serif"
plt.rcParams["font.monospace"] = "Computer Modern Sans Serif"
plt.rcParams["figure.max_open_warning"] = False

ModuleNotFoundError: No module named 'fitting_tools.experiment_generator'

In [ ]:
path = os.path.expandvars("$HOME/NEXT/pdata/{background}/{region}/")

backgrounds = ["208Tl", "214Bi"]
month       = 30*24*3600
year        = 12*month

In [ ]:
time = 50 * year
data = generate_background_experiment(time, table_filename, path, backgrounds=["214Bi", "208Tl"])
data = create_unique_event_number(data, index_name="pevent")

print("Experiment summary")
print("------------------")
data.groupby(["background", "region"]).event.nunique()

In [6]:
# avoid confusion, notice that a cut on tracks would bias this parameter
data = data.drop("ntracks", axis=1)

## Spurious tracks

In [7]:
de = 5e-3
bins = np.arange(0, 3, de)

plt.figure()
h, _ = np.histogram(data["energy"], bins=bins)
plt.stairs(h, bins, label="total", linestyle="--")
plt.ylim([0, None])
plt.xlabel("track energy (MeV)")
plt.yscale("linear")
plt.legend()


plt.figure()
xbins = np.arange(0, 50, 0.1)
ybins = np.arange(0, 10, 1)
plt.hist2d(data["energy"]*1e3, data["nvoxels"], bins=[xbins, ybins]);
plt.xlabel("track energy (keV)")
plt.ylabel("nvoxels")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'nvoxels')

In [8]:
# spurious tracks
data = data.loc[data["energy"]>20e-3]
data.groupby(["background", "region"]).pevent.nunique()

background  region           
208Tl       EP_COPPER_PLATE        238
            INTERNAL_PMT_BASE    85359
            LIGHT_TUBE           33415
            OPTICAL_PAD           6359
            SAPPHIRE_WINDOW       2789
            SIPM_BOARD           24014
            TP_COPPER_PLATE        144
214Bi       EP_COPPER_PLATE         34
            INTERNAL_PMT_BASE     2257
            LIGHT_TUBE           23777
            OPTICAL_PAD            726
            SAPPHIRE_WINDOW        615
            SIPM_BOARD            5596
            TP_COPPER_PLATE         14
Name: pevent, dtype: int64

## 1S2

In [9]:
dn = 1
bins = np.arange(-0.5, 6.5, dn)

plt.figure()
h, _ = np.histogram(data.groupby("event").peak.nunique(), bins=bins, density=True)
plt.stairs(h, bins, label="total", linestyle="--")

for background in backgrounds:
    sel  = (data["background"]==background)
    h, _ = np.histogram(data.loc[sel].groupby("pevent").peak.nunique(), bins=bins, density=True)
    plt.stairs(h, bins, label=background)
        
plt.ylim([0, None])
plt.xlabel("nS2s")
plt.xticks(bins.astype(int))
plt.yscale("linear")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# 1 peak cut
sel_1peak = (data.groupby("pevent").peak.nunique() == 1)
data = data.set_index("pevent").loc[sel_1peak].reset_index()
data.groupby(["background", "component", "region"]).event.nunique()

background  component       region           
208Tl       Body            LIGHT_TUBE           13922
            Energy Plane    EP_COPPER_PLATE        128
                            INTERNAL_PMT_BASE    48289
                            OPTICAL_PAD           3460
                            SAPPHIRE_WINDOW       1486
            Tracking Plane  SIPM_BOARD           10144
                            TP_COPPER_PLATE         79
214Bi       Body            LIGHT_TUBE           14797
            Energy Plane    EP_COPPER_PLATE         13
                            INTERNAL_PMT_BASE      960
                            OPTICAL_PAD            329
                            SAPPHIRE_WINDOW        265
            Tracking Plane  SIPM_BOARD            2279
                            TP_COPPER_PLATE          4
Name: event, dtype: int64

In [11]:
de = 5e-3
bins = np.arange(0, 3, de)

plt.figure()
h, _ = np.histogram(data.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total", linestyle="--")

for background in backgrounds:
    sel  = (data["background"]==background)
    h, _ = np.histogram(data.loc[sel].groupby("pevent").energy.sum(), bins=bins)
    plt.stairs(h, bins, label=background)
    
plt.axvline(x=2.458, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="Qbb")    
plt.ylim([0, None])
plt.xlabel("Total event energy (MeV)")
plt.yscale("linear")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Position variables

In [12]:
dz = 2
bins = np.arange(0, 1300, dz)

plt.figure()
h, _ = np.histogram(data["z"], bins=bins)
plt.stairs(h, bins, label="total")

z = 15
plt.axvline(x=z, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label=f"z={z}")   
plt.xlabel("Track mean z (mm)")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
dr = 2
bins = np.arange(0, 500, dr)

plt.figure()
h, _ = np.histogram(data["r"], bins=bins)
plt.stairs(h, bins, label="total")

r = 460
plt.axvline(x=r, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label=f"r={r} mm")   
plt.xlabel("Track mean radius (mm)")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
dr = 2
bins = np.arange(0, 500, dr)

plt.figure()
h, _ = np.histogram(data["rmin"], bins=bins)
plt.stairs(h, bins, label="total")

r = 460
plt.axvline(x=r, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label=f"r={r} mm")   
plt.xlabel("Track min radius (mm)")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
dr = 2
bins = np.arange(0, 500, dr)

plt.figure()
h, _ = np.histogram(data["rmax"], bins=bins)
plt.stairs(h, bins, label="total")

plt.xlabel("Track max radius (mm)")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Z Fiducial (1S1 equivalent)

In [16]:
sel_z = data["z"]<15
sel_events_z = data[sel_z].set_index("pevent").index.unique()
df = data.set_index("pevent").loc[sel_events_z].reset_index()

sel = df  .groupby(["background", "component", "region"]).event.nunique()
tot = data.groupby(["background", "component", "region"]).event.nunique()
100*sel/tot

background  component       region           
208Tl       Body            LIGHT_TUBE            0.502801
            Energy Plane    EP_COPPER_PLATE       0.613497
                            INTERNAL_PMT_BASE     0.254474
                            OPTICAL_PAD           0.264706
                            SAPPHIRE_WINDOW            NaN
            Tracking Plane  SIPM_BOARD            4.761435
                            TP_COPPER_PLATE       2.247191
214Bi       Body            LIGHT_TUBE            6.084727
            Energy Plane    EP_COPPER_PLATE            NaN
                            INTERNAL_PMT_BASE     0.302725
                            OPTICAL_PAD           0.303951
                            SAPPHIRE_WINDOW            NaN
            Tracking Plane  SIPM_BOARD           13.426942
                            TP_COPPER_PLATE      33.333333
Name: event, dtype: float64

In [17]:
de = 10e-3
bins = np.arange(0, 1.7, de)

plt.figure()
h, _ = np.histogram(df.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total-zcut")
h, _ = np.histogram(data.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total")

plt.axvline(x=2.458, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="Qbb")    
plt.ylim([0, None])
plt.xlabel("Total event energy (MeV)")
plt.yscale("linear")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
dn = 1
bins = np.arange(-0.5, 6.5, dn)

plt.figure()
h, _ = np.histogram(data.groupby("pevent").track.nunique(), bins=bins, density=True)
plt.stairs(h, bins, label="total", linestyle="--")

h, _ = np.histogram(df.groupby("pevent").track.nunique(), bins=bins, density=True)
plt.stairs(h, bins, label="total-zcut", linestyle="--")

for background in backgrounds:
    sel  = (df["background"]==background)
    h, _ = np.histogram(df[sel].groupby("pevent").track.nunique(), bins=bins, density=True)
    plt.stairs(h, bins, label=background + "-z-cut")

plt.xlabel("N tracks")
plt.yscale("linear")
plt.xticks(bins.astype(int))
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plt.figure()
plt.hist2d(df["x"], df["y"], bins=100);
plt.xlabel("X (mm)")
plt.ylabel("Y (mm)");

dr = 2
bins = np.arange(-0.5, 550, dr)

plt.figure()
h, _ = np.histogram((data["x"]**2 + data["y"]**2)**0.5, bins=bins, density=False)
plt.stairs(h, bins, label="total")
h, _ = np.histogram((df["x"]**2 + df["y"]**2)**0.5, bins=bins, density=False)
plt.stairs(h, bins, label="total-zcut")
r = 470
plt.axvline(x=r, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="r=490")   
plt.xlabel("R (mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# apply zcut. apply in whole event instead on tracks?
data = data[data["z"]>=15]

In [25]:
de = 5e-3
bins = np.arange(0, 3, de)

plt.figure()
h, _ = np.histogram(data.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total", linestyle="--")

for background in backgrounds:
    sel  = (data["background"]==background)
    h, _ = np.histogram(data.loc[sel].groupby("pevent").energy.sum(), bins=bins)
    plt.stairs(h, bins, label=background)
    
plt.axvline(x=2.458, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="Qbb")    
plt.ylim([0, None])
plt.xlabel("Total event energy (MeV)")
plt.yscale("linear")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
dz = 3
bins = np.arange(0, 1300, dz)

plt.figure()
h, _ = np.histogram(data["z"], bins=bins)
plt.stairs(h, bins, label="total")

plt.xlabel("Track mean z (mm)")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Rmin weird peak

In [26]:
dr = 3
bins = np.arange(-0.5, 500, dr)

plt.figure()
h, _ = np.histogram(data["rmin"], bins=bins, density=False)
plt.stairs(h, bins, label="total")

r = 460
plt.axvline(x=r, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label=f"r={r} mm")   
plt.xlabel("R min (mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
sel_rmin = data["rmin"]>460
sel_events_rmin = data[sel_rmin].set_index("pevent").index.unique()
df = data.set_index("pevent").loc[sel_events_rmin].reset_index()

sel = df  .groupby(["background", "component", "region"]).event.nunique()
tot = data.groupby(["background", "component", "region"]).event.nunique()
100*sel/tot

background  component       region           
208Tl       Body            LIGHT_TUBE            3.499531
            Energy Plane    EP_COPPER_PLATE       1.851852
                            INTERNAL_PMT_BASE     1.059726
                            OPTICAL_PAD           1.208726
                            SAPPHIRE_WINDOW       1.354839
            Tracking Plane  SIPM_BOARD            1.340621
                            TP_COPPER_PLATE            NaN
214Bi       Body            LIGHT_TUBE            7.813285
            Energy Plane    EP_COPPER_PLATE            NaN
                            INTERNAL_PMT_BASE     1.720648
                            OPTICAL_PAD           0.304878
                            SAPPHIRE_WINDOW       0.754717
            Tracking Plane  SIPM_BOARD            1.664145
                            TP_COPPER_PLATE      33.333333
Name: event, dtype: float64

In [28]:
dn = 1
bins = np.arange(-0.5, 6.5, dn)

plt.figure()
h, _ = np.histogram(data.groupby("event").track.nunique(), bins=bins, density=True)
plt.stairs(h, bins, label="total", linestyle="--")

# h, _ = np.histogram(df.groupby("event").track.nunique(), bins=bins, density=True)
# plt.step(bins[1:], h, label="total-rmincut", linestyle="--")

for background in backgrounds:
    sel  = (df["background"]==background)
    h, _ = np.histogram(df[sel].groupby("event").track.nunique(), bins=bins, density=True)
    plt.stairs(h, bins, label=background + "-rmin-cut")

plt.xlabel("N tracks")
plt.yscale("linear")
plt.xticks(bins.astype(int))
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
de = 10e-3
bins = np.arange(0, 3.0, de)

plt.figure()
h, _ = np.histogram(df.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total-rmin-cut")
h, _ = np.histogram(data.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total")

plt.axvline(x=2.458, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="Qbb")    
plt.ylim([0, None])
plt.xlabel("Total event energy (MeV)")
plt.yscale("linear")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
rcut = 460

plt.figure()
plt.hist2d(df["x"], df["y"], bins=100);
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(rcut*np.cos(theta), rcut*np.sin(theta), c="r")
plt.xlabel("X (mm)")
plt.ylabel("Y (mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
dr = 2
bins = np.arange(-0.5, 550, dr)

plt.figure()
h, _ = np.histogram(data["r"], bins=bins, density=False)
plt.stairs(h, bins, label="total")
h, _ = np.histogram(df["r"], bins=bins, density=False)
plt.stairs(h, bins, label="total-rmin")
plt.axvline(x=rcut, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="r=490")   
plt.xlabel("R max (mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
# apply the cut
sel_rmin = data["rmin"]<460
sel_events_rmin = data[sel_rmin].set_index("pevent").index.unique()
data = data.set_index("pevent").loc[sel_events_rmin].reset_index()

## E<2.0 MeV ??

In [33]:
de = 5e-3
bins = np.arange(0, 3, de)

plt.figure()
h, _ = np.histogram(data.groupby("pevent").energy.sum(), bins=bins)
plt.stairs(h, bins, label="total", linestyle="--")

for background in backgrounds:
    sel  = (data["background"]==background)
    h, _ = np.histogram(data.loc[sel].groupby("pevent").energy.sum(), bins=bins)
    plt.stairs(h, bins, label=background)
    
plt.axvline(x=2.458, ymin=0, ymax=1, linestyle="--", linewidth=1, color="r", label="Qbb")    
plt.ylim([0, None])
plt.xlabel("Total event energy (MeV)")
plt.yscale("linear")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
energy = data.groupby("pevent").energy.sum()
sel = energy[energy<2.0]
df = data.set_index("pevent").loc[sel.index.unique()].reset_index()

In [39]:
sel = df  .groupby(["background", "component", "region"]).event.nunique()
tot = data.groupby(["background", "component", "region"]).event.nunique()
100 * sel/tot

background  component       region           
208Tl       Body            LIGHT_TUBE            7.005127
            Energy Plane    EP_COPPER_PLATE       2.469136
                            INTERNAL_PMT_BASE     2.966190
                            OPTICAL_PAD           3.951637
                            SAPPHIRE_WINDOW       2.973497
            Tracking Plane  SIPM_BOARD            4.715229
                            TP_COPPER_PLATE       5.747126
214Bi       Body            LIGHT_TUBE           24.242880
            Energy Plane    EP_COPPER_PLATE            NaN
                            INTERNAL_PMT_BASE     3.238866
                            OPTICAL_PAD           2.743902
                            SAPPHIRE_WINDOW       3.396226
            Tracking Plane  SIPM_BOARD            7.824331
                            TP_COPPER_PLATE      33.333333
Name: event, dtype: float64

In [40]:
rcut = 460

plt.figure()
plt.hist2d(df["x"], df["y"], bins=100);
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(rcut*np.cos(theta), rcut*np.sin(theta), c="r")
plt.xlabel("X (mm)")
plt.ylabel("Y (mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Events reconstructed with E<2.0 MeV corresponds to events at the borders, whose energy is degraded due to the smaller light collection at these extreme region (the light-table is much lower at the edges). This effect should be corrected by Kr map!

In [63]:
# remove from E<2.0 MeV from data and check radial distribution
sel = np.setdiff1d(data.pevent.unique(), df.pevent.unique())

sel = data.set_index("pevent").loc[sel]

rcut = 460

plt.figure()
plt.hist2d(sel["x"], sel["y"], bins=100);
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(rcut*np.cos(theta), rcut*np.sin(theta), c="r")
plt.xlabel("X (mm)")
plt.ylabel("Y (mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Conclusion**:

The E>2.0 MeV removes not only degraded energy events due to worse border collection efficiency, but also takes off the z<15 mm no-S1 events (since I set $s1nmin=0$ in the processing). Why the degradation? Check/Improve Kr map

In [15]:
df = data

In [16]:
background = "214Bi"
region     = "SIPM_BOARD"

selected_events = df[(df["background"]==background) & (df["region"] == region)]

In [17]:
selected_events

,pevent,event,background,region,peak,track,out_of_map,nvoxels,nhits,length,...,yb1,zb1,eb1,xb2,yb2,zb2,eb2,ovlp_e,nfile,component
19527,94264,16240604,214Bi,SIPM_BOARD,0,0,False,5,2124,19.000000,...,138.709344,4.066629,0.293169,102.795644,138.041675,4.652057,0.289461,0.289311,101,Tracking Plane
19528,87511,16240614,214Bi,SIPM_BOARD,0,0,False,108,10002,198.360278,...,225.158760,896.458839,0.761956,358.715346,311.452777,848.367283,0.153109,0.000000,101,Tracking Plane
19529,115968,16240620,214Bi,SIPM_BOARD,0,0,False,101,6400,190.035962,...,324.446132,624.492953,0.869056,-120.721077,317.460283,468.894390,0.033150,0.000000,101,Tracking Plane
19530,115968,16240620,214Bi,SIPM_BOARD,0,1,False,13,2132,26.145973,...,296.244666,472.400537,0.636701,-167.208739,281.610423,472.053092,0.636102,0.628525,101,Tracking Plane
19531,165528,16240624,214Bi,SIPM_BOARD,0,0,False,103,9613,277.725830,...,181.712262,522.490643,0.856522,-211.631316,49.809357,552.321046,0.145400,0.000000,101,Tracking Plane
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22413,157720,18974214,214Bi,SIPM_BOARD,0,0,False,119,9834,214.982271,...,-119.393249,212.951994,0.576943,155.679146,-93.777365,69.425743,0.138223,0.000000,118,Tracking Plane
22414,107349,18974222,214Bi,SIPM_BOARD,0,0,False,126,10197,243.816205,...,4.847559,159.176740,0.780156,6.236299,5.597114,32.464625,0.074721,0.000000,118,Tracking Plane
22415,69674,18974244,214Bi,SIPM_BOARD,0,0,False,76,8186,161.729404,...,230.413300,183.044149,0.822928,-163.219459,302.601331,134.863768,0.082726,0.000000,118,Tracking Plane
22416,69674,18974244,214Bi,SIPM_BOARD,0,1,False,9,1513,22.819405,...,371.035008,147.013397,0.495516,-147.045498,377.062627,146.971422,0.495516,0.495516,118,Tracking Plane


In [18]:
event = np.random.choice(selected_events.event.unique())
print("Event:", event)

sample = df[df["event"]==event]

# sample = selected_events.sample(1)

ntracks = sample.track.nunique()
event   = sample.event.values[0]
nfile   = sample.nfile.values[0]

Event: 16562238


In [29]:
filename = (path.format(background=background, region=region)
                .replace("/pdata/pdata_*", f"/beersheba/beersheba_{nfile}_{background}"))

eventMap    = load_dst(filename, "Run", "eventMap").set_index("evt_number")

mchits      = load_dst(filename,  "MC", "hits")     .set_index("event_id").loc[eventMap.loc[event]]
mcparticles = load_dst(filename,  "MC", "particles").set_index("event_id").loc[eventMap.loc[event]]

deco = pd.read_hdf(filename, "DECO/Events").set_index("event").loc[event]

AttributeError: 'NoneType' object has no attribute 'set_index'

In [43]:
mcparticles.loc[:, ("particle_name", "particle_id", "kin_energy",
                    "creator_proc", "final_proc", "mother_id")].sort_values("particle_id")

,particle_name,particle_id,kin_energy,creator_proc,final_proc,mother_id
event_id,,,,,,
3698370,Bi214,1,0.000000,none,RadioactiveDecayBase,0
3698370,Po214,2,0.000033,RadioactiveDecayBase,RadioactiveDecayBase,1
3698370,anti_nu_e,3,0.533635,RadioactiveDecayBase,Transportation,1
3698370,e-,4,2.736132,RadioactiveDecayBase,eIoni,1
3698370,e-,5,0.053873,eIoni,eIoni,4
3698370,e-,6,0.072742,eIoni,eIoni,4
3698370,e-,7,0.107021,eIoni,eIoni,4
3698370,e-,8,0.038209,eIoni,eIoni,4
3698370,gamma,9,0.235864,eBrem,phot,4


In [44]:
plt.figure()
plt.scatter(mchits["x"], mchits["y"], s=10)
plt.scatter(deco["X"], deco["Y"], s=10, alpha=0.1)

r=492
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(r*np.cos(theta), r*np.sin(theta), c="k")

r=400
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(r*np.cos(theta), r*np.sin(theta), c="r")
plt.xlim([-500, 500])
plt.ylim([-500, 500]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
plt.figure()
plt.scatter(mchits["z"], mchits["y"], s=10)
plt.scatter(deco["Z"], deco["Y"], s=10, alpha=0.1)

plt.ylim([-500, 500]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
filename = (path.format(background=background, region=region)
                .replace("/pdata/pdata_*", f"/penthesilea/penthesilea_{nfile}_{background}"))

reco = pd.read_hdf(filename, "RECO/Events").set_index("event").loc[event]

In [53]:
plt.figure()
plt.scatter(mchits["z"], mchits["y"], s=10)
plt.scatter(reco["Z"], reco["Y"], s=10, alpha=0.1)

plt.ylim([-500, 500]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
filename = (path.format(background=background, region=region)
                .replace("/pdata/pdata_*", f"/detsim/detsim_{nfile}_{background}"))

with tb.open_file(filename) as h5file:
    evts = h5file.root.Run.events.read()["evt_number"]
    idx = np.argwhere(evts == event).flatten()[0]
    pmtwfs = h5file.root.pmtrd[idx]
    
    
filename = (path.format(background=background, region=region)
                .replace("/pdata/pdata_*", f"/hypathia/hypathia_{nfile}_{background}"))

S1 = pd.read_hdf(filename, "PMAPS/S1").set_index("event")
S2 = pd.read_hdf(filename, "PMAPS/S2").set_index("event").loc[event]

In [105]:
plt.figure()

times = np.arange(0, pmtwfs.shape[-1])*25/1000
plt.plot(times, np.sum(pmtwfs, axis=0)*40)

plt.scatter(S2.time/1e3, S2.ene, s=20, c="r")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …